In [1]:
import sys
sys.path.append('C:\\Users\\monty.minh\\Documents\\Optimizer\\')

from sympy import Matrix

from model.data import *
from model.preprocessing import *
from model.optimization import *

Define a file path to the Excel input file
Define a file path to save the Excel output file

Data.filepath = 
Results.save_location = 


In [2]:
class Results:
    volume, cost = [], []

In [3]:
Data.filepath = 'C:/Users/monty.minh/Documents/Optimizer/0. Inputs/Inputs Example.xlsm'
Results.save_location = 'C:/Users/monty.minh/Documents/Optimizer/4. Results/Results.xlsx'

get_timeframe()

for yr in range(Data.timeframe[0], Data.timeframe[1]):
    Data.year = yr

    preprocess()
    optimize()
    #unpack_results()

In [4]:
from model import *
from model.data import Data, Results


def postprocess():
    """
    Unpack the program results into a volume vector
    and a cost vector.

    Input from data.py
    ------------------
    Data.linear_program: scipy.optimize.optimize.OptimizeResult
        Result of the linear program.

    Outputs to data.py
    ------------------
    Results.volume: list
        List of customers' volumes by optimization instances
    Results.cost: list
        List of customers' costs by optimization instances
    Results.split: int
        Index to split the above vector into inbound / outbound
    """
    # Unpack Volume
    Results.volume.append(Data.linear_program[:, np.newaxis])

    # Unpack Cost
    Results.cost.append(
        (Data.linear_program * Data.objective_vector)[:, np.newaxis])

    # Collect split index, to remove Data dependencies.
    # Next step is to free up all the memory from Data.
    Results.split = Data.dimF


def save_to_excel():
    """
    Method to save volume and cost data to Excel file

    Inputs from data.py
    -------------------
    Results.save_location: str (filepath)
        Path to save output Excel file
    Results.volume: list
        List of customers' volume by optimization instances
    Results.cost: list
        List of customers' cost by optimization instances
    """

    def quick_save(df, name): return df.to_excel(
        writer, name, index=False, header=True, startrow=0, startcol=0)

    # Generate the inbound template
    inbound_prefix = pd.DataFrame(data=np.vstack([
        np.hstack([Data.factory_names[prod] for prod in Data.product_list]),
        np.hstack([[prod for elem in Data.factory_names[prod]]
                   for prod in Data.product_list])
    ]).T,
        columns=['Factory', 'Product'])

    # Generate the outbound template
    df = pd.read_excel(Data.filepath,
                       sheet_name="Customer List")
    outbound_prefix = pd.DataFrame(data=np.vstack([
        np.repeat(df[df['Sales Product'] == prod][[
            'Customer ID', 'Sales Product', 'Province'
        ]].to_numpy(),
            repeats=Data.factory_sizes[prod],
            axis=0) for prod in Data.product_list
    ]),
        columns=['ID', 'Product', 'Province'])
    
    outbound_prefix['Factory'] = np.hstack([
        Data.factory_names[prod] * Data.customer_sizes[prod]
        for prod in Data.product_list
    ])

    # Calculate the list of years
    Results.years = np.arange(Data.timeframe[0], Data.timeframe[1]+1)

    with pd.ExcelWriter(Results.save_location) as writer:

        # Inbound Cost
        df = inbound_prefix.copy()
        df[Results.years] = np.hstack(
            Results.cost)[:Results.split]
        df = df[df[Results.years].sum(axis=1) != 0]
        quick_save(df, 'Inbound Cost Per Factory')
        
        # Outbound Cost
        df = outbound_prefix.copy()
        # Concatenate with the volume per year
        df[Results.years] = np.hstack(
            Results.cost)[Results.split:]
        # Remove all zeros rows
        df = df[df[Results.years].sum(axis=1) != 0]
        quick_save(df, 'Outbound Cost Per Customer')
        
        # Inbound Volume
        df = inbound_prefix.copy()
        df[Results.years] = np.hstack(
            Results.volume)[:Results.split]
        df = df[df[Results.years].sum(axis=1) != 0]
        quick_save(df, 'Inbound Volume Per Factory')

        # Outbound Volume
        df = outbound_prefix.copy()
        # Concatenate with the volume per year
        df[Results.years] = np.hstack(
            Results.volume)[Results.split:]
        # Remove all zeros rows
        df = df[df[Results.years].sum(axis=1) != 0]
        quick_save(df, 'Outbound Volume Per Customer')


def free_memory():
    """Free memory by deleting some Data attributes"""

    # Free up memory
    keep = ["filepath", "factory_sizes", 
            "customer_sizes", "factory_names", 
            "product_list", "timeframe"]
    _ = [
        delattr(Data, attr) for attr in dir(Data)
        if (attr[:2] != '__') and (attr not in keep)
    ]
    del _
    gc.collect()


In [5]:
postprocess()

In [6]:
Results.years = [2022]

In [7]:
# Generate the inbound template
inbound_prefix = pd.DataFrame(data=np.vstack([
    np.hstack([Data.factory_names[prod] for prod in Data.product_list]),
    np.hstack([[prod for elem in Data.factory_names[prod]]
               for prod in Data.product_list])
]).T,
    columns=['Factory', 'Product'])

In [8]:
df = inbound_prefix.copy()
df[Results.years] = np.hstack(
    Results.volume)[:Results.split]
#df = df[df[Results.years].sum(axis=1) != 0]

In [38]:
# Generate the outbound template
df = pd.read_excel(Data.filepath,
                   sheet_name="Customer List")

outbound_prefix = pd.DataFrame(data=np.vstack([
    np.repeat(df[df['Sales Product'] == prod][[
        'Customer ID', 'Sales Product', 'Province'
    ]].to_numpy(),
        repeats=Data.factory_sizes[prod],
        axis=0) for prod in Data.product_list
]),
    columns=['ID', 'Product', 'Province'])

outbound_prefix['Factory'] = np.hstack([
    Data.factory_names[prod] * Data.customer_sizes[prod]
    for prod in Data.product_list
])

outbound_prefix = outbound_prefix.sort_values(['Product', 'Factory'])

In [39]:
df = outbound_prefix.copy()
# Concatenate with the volume per year
df[Results.years] = np.hstack(
    Results.volume)[Results.split:]
# Remove all zeros rows
#df = df[df[Results.years].sum(axis=1) != 0]

In [15]:
Data.objective_vector

array([ 50.,  40.,  30., 100.,  30.,  40.,  60.,  90.,  60.,  70.])

In [40]:
inbound_prefix

,Factory,Product
0,Binh Dinh,Bag
1,Hai Phong,Bag
2,Binh Dinh,Bulk


In [28]:
Matrix(
    np.vstack([
        np.hstack([Data.objective_vector, np.zeros(1)]),
        np.hstack([Data.constraints_matrix, Data.constraints_vector]),
        np.hstack([Data.linear_program, np.dot(Data.constraints_vector.T, 
                                               Data.objective_vector)])
    ])
)

Matrix([
[50.0, 40.0, 30.0, 100.0, 30.0, 40.0, 60.0, 90.0, 60.0, 70.0,       0],
[   0,    0,    0,  -1.0,    0, -1.0,    0,    0,    0,    0,   -10.0],
[   0,    0,    0,     0, -1.0,    0, -1.0,    0,    0,    0,   -20.0],
[   0,    0,    0,     0,    0,    0,    0, -1.0,    0,    0,   -10.0],
[   0,    0,    0,     0,    0,    0,    0,    0, -1.0,    0,   -20.0],
[   0,    0,    0,     0,    0,    0,    0,    0,    0, -1.0,   -15.0],
[   0,    0,    0,   1.0,  1.0,    0,    0,  1.0,  1.0,  1.0,   150.0],
[   0,    0,    0,     0,    0,  1.0,  1.0,    0,    0,    0,    20.0],
[   0,    0,    0,     0,    0,    0,    0,  1.0,  1.0,  1.0,   120.0],
[-1.0,    0, -1.0,   1.0,  1.0,    0,    0,  1.0,  1.0,  1.0,       0],
[   0, -1.0,    0,     0,    0,  1.0,  1.0,    0,    0,    0,       0],
[   0, 10.0, 65.0,     0, 20.0, 10.0,    0, 10.0, 20.0, 15.0, 13950.0]])

In [250]:
Results.years = np.arange(2021, 2021+ len(Results.cost))

In [1]:
outbound_prefix = pd.DataFrame(data = np.vstack([
    np.repeat(df[df['Sales Product'] == prod][['Customer ID', 'Sales Product','Province']].to_numpy(),
          repeats=Data.factory_sizes[prod], axis = 0
             ) for prod in Data.product_list]), 
             columns = ['ID', 'Product', 'Province']
            )

outbound_prefix['Factory'] = np.hstack([
    Data.factory_names[prod] * Data.customer_sizes[prod]
    for prod in Data.product_list
])

df = outbound_prefix.copy()
df[Results.years] = np.hstack(Results.volume)[Results.split:]

df = outbound_prefix.copy()
df[Results.years] = np.hstack(Results.cost)[Results.split:]

NameError: name 'pd' is not defined